In [55]:
import pandas as pd
import numpy as np
#import warnings

In [56]:
#warnings.filterwarnings('ignore')
pd.set_option('display.max_rows', 900)
pd.set_option('display.max_columns', 900)
pd.set_option('display.width', 1000)

In [57]:
def add_new_columns(state_df):
    state_df['sum_r12'] = ''
    state_df['sum_r345'] = ''
    state_df['relocate'] = ''
    return state_df

In [58]:
df = pd.read_csv("data/homes_example.csv")
#df

In [59]:
#df = df.drop(columns=['Count of FACID']) 
df = df.fillna(method='ffill')
df['Comp5star'] = df['Comp5star'].astype(int)
df['PROVNUM']   = df['PROVNUM'].astype(str)
df['PROVNAME']  = df['PROVNAME'].astype(str)

#df.head()

In [60]:
def process_county(county_df):
    # check if rating of 1 or 2 does not exist first
    ratings = county_df['Comp5star']
    #print("ratings:",ratings)
    
    if (1 not in ratings.values) and (2 not in ratings.values):
        #print("NO 1 and NO 2")
        county_df.loc[0:0,'relocate'] = 'N/A'
        return county_df
    else:
        #print("PROCESSING COUNTY:",county_df['County_name'][0])
        
        # ------- get indicies of '1s and 2s'
        i_1_2 = ratings.isin([1,2])
        i_1_2 = i_1_2[i_1_2].index
                
        sum_1_2 = 0
        for i in i_1_2:
            #print("idx_of_1_or_2:", i, "->", county_df['Sum of MDScensus_avg'][i])
            sum_1_2 =  sum_1_2 + county_df['Sum of MDScensus_avg'][i]
            
        #print("sum of 1s & 2s:", sum_1_2)
        # insert sum into same line as name of the county
        county_df.loc[0, 'sum_r12'] = sum_1_2
        
        # ------ get indicies of '3s, 4s, and 5s'
        i_3_4_5 = ratings.isin([3,4,5])
        i_3_4_5 = i_3_4_5[i_3_4_5].index
                
        sum_3_4_5 = 0
        for i in i_3_4_5:
            #print("idx_of_3_or_4_or_5:", i, "->", county_df['Sum of Beds_Avail'][i])
            sum_3_4_5 =  sum_3_4_5 + county_df['Sum of Beds_Avail'][i]
            
        #print("sum of 3s & 4s & 5s:", sum_3_4_5)
        # insert sum into same line as name of the county
        county_df.loc[0, 'sum_r345'] = sum_3_4_5
        
        if sum_1_2 > sum_3_4_5:
            # mark each home that is unable to relocate
            for i in i_1_2:
                county_df.loc[i, 'relocate'] = 'NO_SPACE'
        
        else:
            # enough space available, 1 & 2 home that can 
            # relocate within the county
            for i in i_1_2:
                county_df.loc[i, 'relocate'] = 'CAN_RELOCATE'              
        
        #print("------------------------------------------------")
        return county_df

In [61]:
l_df_out = []
states = df.STATE.unique()

for state in states:
    #print("/////////////////////////////////////////", state, "/////////////////////////////////")
    state_df = df.loc[df['STATE'] == state]
    state_df = add_new_columns(state_df.copy())
    #print(state_df)
    
    # get all counties for this state
    counties_in_state = state_df.County_name.unique()
    #print(counties_in_state)

    for county in counties_in_state:
        #print("-----------------------------------------", county, "------------------------------")
        county_df = state_df.loc[state_df['County_name'] == county]
        
        county_processed_df = process_county(county_df.reset_index())
        #print(county_processed_df)
        
        l_df_out.append(county_processed_df)
        #print("-----------------------------------------", county, "------------------------------")
    #print("/////////////////////////////////////////", state, "/////////////////////////////////\n")

df_out = (pd.concat(l_df_out)).reset_index(drop=True)
df_out.to_csv("homes_df_out.csv", index=False)
#df_out.head()